## content based filtering

In [1]:
#https://www.kaggle.com/gorochu/complete-imdb-movies-dataset?select=movie.csv

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('total_imdb.csv')

In [4]:
df.shape

(62058, 16)

In [5]:
# df.info()

In [6]:
# df.isnull().sum()

In [7]:
movies = df[['title','year','users_rating','votes','actors','genre' ,'description','directors','imdb_url']].copy()

In [8]:
df.iloc[0].imdb_url  

'https://www.imdb.com/title/tt7139936/'

In [9]:
movies.dropna(inplace=True)

In [10]:
movies.head(2)

,title,year,users_rating,votes,actors,genre,description,directors,imdb_url
0,A Rainy Day in New York,2019.0,6.6,"21,903","['Timothée Chalamet', 'Elle Fanning', 'Liev Sc...","['Comedy', 'Romance']",A young couple arrives in New York for a weeke...,['Woody Allen'],https://www.imdb.com/title/tt7139936/
1,Murder Manual,2020.0,2.4,192,"['Emilia Clarke', 'Hadley Fraser', 'Sylvia Pan...","['Horror', 'Thriller']","Creepy, terrifying chapters from our book of h...","['Michael Escobedo', 'Kelly Hallmark', '4 more...",https://www.imdb.com/title/tt12384178/


In [11]:
movies = movies[movies['year'] >= 1994].copy()

In [12]:
movies.shape

(34473, 9)

In [13]:
import re

In [14]:
movies['votes'] = movies['votes'].apply(lambda x: int(str(x).replace(',','')))

In [15]:
min = movies['votes'].quantile(.85)
print(min)
new_df = movies[movies['votes'] >= min].copy()

7514.600000000002


In [16]:
new_df.shape

(5171, 9)

In [17]:
min = movies['votes'].quantile(.85)
C = movies['users_rating'].mean()

def score_it(row):
    v = row['votes']
    avg = row['users_rating']
    return round((v/(v+min) * avg) + (min/(v+min) * C), 2)

In [18]:
new_df['score'] = new_df.apply(score_it,axis=1)

In [19]:
new_df.sort_values(by='score',ascending=False,inplace=True)
new_df.reset_index(inplace=True)

In [20]:
new_df.head(2)

,index,title,year,users_rating,votes,actors,genre,description,directors,imdb_url,score
0,53985,The Shawshank Redemption,1994.0,9.3,2250362,"['Tim Robbins', 'Morgan Freeman', 'Bob Gunton'...",['Drama'],Two imprisoned men bond over a number of years...,['Frank Darabont'],https://www.imdb.com/title/tt0111161/,9.29
1,53968,The Dark Knight,2008.0,9.0,2218088,"['Christian Bale', 'Heath Ledger', 'Aaron Eckh...","['Action', 'Crime', 'Drama', 'Thriller']",When the menace known as the Joker wreaks havo...,['Christopher Nolan'],https://www.imdb.com/title/tt0468569/,8.99


In [21]:
new_df['imdb_id'] = new_df['imdb_url'].apply(lambda x: x.split('/')[-2])

In [22]:
import ast

In [23]:
ast.literal_eval(new_df['genre'][3])

['Action', 'Adventure', 'Sci-Fi', 'Thriller']

In [24]:
def convert(x):
    l =[]
    for i in ast.literal_eval(str(x)):
        l.append(i.replace("'",""))
    return l

def convertf(x):
    l =[]
    for i in ast.literal_eval(str(x)):
        l.append(i.replace("'",""))
        if len(l) == 4:
            return l
    return l

In [25]:
new_df['genre'] = new_df['genre'].apply(convert)
new_df['actors'] = new_df['actors'].apply(convertf)

In [26]:
def convertd(x):
    l =[]
    for i in x.split():
        l.append(i)
    return l

In [27]:
new_df['description'] = new_df['description'].apply(convertd)

In [28]:
new_df['tags'] = new_df['description'] + new_df['genre'] + new_df['actors']

In [29]:
df2 = new_df[['title','year','imdb_id','genre','score','tags']].copy()

#### final df

In [30]:
df2.head(20)

,title,year,imdb_id,genre,score,tags
0,The Shawshank Redemption,1994.0,tt0111161,[Drama],9.29,"[Two, imprisoned, men, bond, over, a, number, ..."
1,The Dark Knight,2008.0,tt0468569,"[Action, Crime, Drama, Thriller]",8.99,"[When, the, menace, known, as, the, Joker, wre..."
2,Pulp Fiction,1994.0,tt0110912,"[Crime, Drama]",8.89,"[The, lives, of, two, mob, hitmen,, a, boxer,,..."
3,Inception,2010.0,tt1375666,"[Action, Adventure, Sci-Fi, Thriller]",8.79,"[A, thief, who, steals, corporate, secrets, th..."
4,Forrest Gump,1994.0,tt0109830,"[Drama, Romance]",8.79,"[The, presidencies, of, Kennedy, and, Johnson,..."
5,Fight Club,1999.0,tt0137523,[Drama],8.79,"[An, insomniac, office, worker, and, a, devil-..."
6,The Matrix,1999.0,tt0133093,"[Action, Sci-Fi]",8.69,"[A, computer, hacker, learns, from, mysterious..."
7,Interstellar,2014.0,tt0816692,"[Adventure, Drama, Sci-Fi, Thriller]",8.58,"[A, team, of, explorers, travel, through, a, w..."
8,Saving Private Ryan,1998.0,tt0120815,"[Drama, War]",8.58,"[Following, the, Normandy, Landings,, a, group..."
9,The Green Mile,1999.0,tt0120689,"[Crime, Drama, Fantasy, Mystery]",8.58,"[The, lives, of, guards, on, Death, Row, are, ..."


In [31]:
import nltk
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

def stem(text):
    y=[]
    for i in text:
        y.append(ps.stem(i.lower()))
    return " ".join(y)

In [32]:
df2['tags'] = df2['tags'].apply(stem)

### vectorization

In [33]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=6864,stop_words='english')

In [34]:
vector = cv.fit_transform(df2['tags']).toarray()

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
sim = cosine_similarity(vector)

In [37]:
similarity_df = pd.DataFrame(sim,index=df2.title,columns=df2.title)

In [38]:
similarity_df

title,The Shawshank Redemption,The Dark Knight,Pulp Fiction,Inception,Forrest Gump,Fight Club,The Matrix,Interstellar,Saving Private Ryan,The Green Mile,...,Saving Christmas,Birdemic: Shock and Terror,Dragonball Evolution,Battlefield Earth,Superbabies: Baby Geniuses 2,House of the Dead,Son of the Mask,Epic Movie,The Hottie & the Nottie,Disaster Movie
title,,,,,,,,,,,,,,,,,,,,,
The Shawshank Redemption,1.000000,0.042601,0.097823,0.000000,0.086711,0.055641,0.000000,0.050063,0.050063,0.042601,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.039344,0.061314,0.000000,0.000000
The Dark Knight,0.042601,1.000000,0.118771,0.072836,0.105279,0.090075,0.109254,0.081044,0.081044,0.103448,...,0.059470,0.079181,0.064651,0.036418,0.000000,0.043769,0.031846,0.000000,0.000000,0.000000
Pulp Fiction,0.097823,0.118771,1.000000,0.000000,0.080582,0.051709,0.083624,0.093048,0.046524,0.118771,...,0.068279,0.000000,0.000000,0.083624,0.000000,0.000000,0.073127,0.000000,0.000000,0.000000
Inception,0.000000,0.072836,0.000000,1.000000,0.037062,0.000000,0.115385,0.213980,0.128388,0.036418,...,0.000000,0.041812,0.170697,0.153846,0.066299,0.092450,0.000000,0.052414,0.000000,0.000000
Forrest Gump,0.086711,0.105279,0.080582,0.037062,1.000000,0.045835,0.074125,0.041239,0.164957,0.175466,...,0.060523,0.000000,0.000000,0.037062,0.000000,0.000000,0.064820,0.000000,0.137505,0.044544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
House of the Dead,0.000000,0.043769,0.000000,0.092450,0.000000,0.000000,0.092450,0.102869,0.102869,0.087538,...,0.000000,0.050252,0.082061,0.092450,0.039841,1.000000,0.000000,0.062994,0.000000,0.055556
Son of the Mask,0.039344,0.031846,0.073127,0.000000,0.064820,0.000000,0.067267,0.000000,0.000000,0.063693,...,0.109847,0.036564,0.119416,0.000000,0.057977,0.000000,1.000000,0.045835,0.041595,0.040423
Epic Movie,0.061314,0.000000,0.000000,0.052414,0.000000,0.000000,0.000000,0.058321,0.000000,0.000000,...,0.042796,0.000000,0.046524,0.052414,0.045175,0.062994,0.045835,1.000000,0.064820,0.062994


In [39]:
# import pickle
# pickle.dump(similarity_df,open('cb_data.pkl','wb'))

In [40]:
def recommend(movie):
    row = similarity_df[movie]
    sort_titles = row.sort_values(ascending=False)
    return sort_titles.iloc[1:6].index.tolist()

In [46]:
recommend('Batman Begins')

['The Dark Knight',
 'The Dark Knight Rises',
 'Batman Ninja',
 'The Legend of Hercules',
 'American Psycho']

In [42]:
# df3 = df2[['title','imdb_id','year','genre','score']]

In [43]:
# import pickle
# pickle.dump(df3,open('5kMeta.pkl','wb'))

In [45]:
titles = df2.title.sort_values(ascending=True)
r = []
def search(name):
    for i in titles:
        if(name.lower() in i.lower()):
            r.append(i)
    r.sort()
    for j in r:
         print(j)
search('batman')

Batman & Robin
Batman Begins
Batman Forever
Batman Ninja
Batman and Harley Quinn
Batman v Superman: Dawn of Justice
Batman: Gotham by Gaslight
Batman: The Killing Joke
The Lego Batman Movie
